## 품질계획 현황 수집

In [17]:
from common import commonFunc as cf
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질계획 현황"
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)
targetData = metadata.loc[metadata.자료명==DATANAME]

preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/품질계획 현황/selectQltPlnList.csv'

In [18]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectQltPlnList'

In [19]:
APIKEYLEN = len(APIKEY)

## 연도별 공사계약 목록에서  SPTNO 활용

In [20]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/연도별 공사계약 목록/selectIoCmProjConstYearContractList.csv", encoding="ms949")

In [21]:
dfCombi = imsiDf.loc[:,["sptno"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)

In [22]:
dfCombiLen = dfCombi.shape[0]
dfCombiLen

1220

## 입력 파라미터 정보 불러오기¶

In [23]:
newParam = dfCombi.sptno.drop_duplicates().tolist()
newParam

['C2004186',
 'C2019022',
 'C2003095',
 'C2003142',
 'C1997009',
 'C2004031',
 'C2003112',
 'C2005069',
 'C2009055',
 'C2012010',
 'C2013937',
 'C2015006',
 'C2015008',
 'C2015009',
 'C2004091',
 'C2012022',
 'C2015007',
 'C2002006',
 'C2003011',
 'C2009042',
 'C2005032',
 'C2015025',
 'C2003012',
 'C2015003',
 'C2002070',
 'C1996005',
 'T2002001',
 'C2012058',
 'C2012059',
 'C2002015',
 'C2002003',
 'C2007038',
 'C1997010',
 'C2007079',
 'C2022008',
 'C2001027',
 'C2012035',
 'C2012060',
 'C2021023',
 'C2006018',
 'C2004003',
 'C2006016',
 'C2016032',
 'C2006017',
 'C2018031',
 'C2009105',
 'C2009104',
 'C2009106',
 'C2017025',
 'C2006006',
 'C2007032',
 'C2007031',
 'C2004128',
 'C2005019',
 'C2011030',
 'C2015034',
 'C2014005',
 'C2016001',
 'C2017018',
 'C2007076',
 'C2017028',
 'C2017030',
 'C2017027',
 'C2017038',
 'C2014014',
 'C2014017',
 'C2003100',
 'C2004042',
 'C2004033',
 'C2008013',
 'C2006048',
 'C2018005',
 'C2009103',
 'C2009046',
 'C2009099',
 'C2009047',
 'C2015012',

## 파라미터정보 불러오기

In [24]:
orgParam = []

try:
    # 기존 데이터 및 파라미터 정보 불러오기
    orgParam = cf.loadparam(SITENAME,DATANAME,SERVICENAME)
## 파일이 없는경우
except Exception as e:
    orgParam = []
    print(e)

sptnoList = list( set(newParam) - set(orgParam) )
# 모드: 0=종료 1=append 2=새로생성
mode = 2

# 업데이트 할 내용이 없으면 종료
if sptnoList == []:
    mode = 0
    print("{} 정보 quit모드 {} ".format(DATANAME, mode))
    quit()
# orgParam == []
elif orgParam == []:
    mode = 2
    print("{} 정보 new 모드 {} ".format(DATANAME, mode))
# 이외에는 append 모드
else:
    mode = 1
    print("{} 정보 append 모드 {} ".format(DATANAME, mode))

[Errno 2] No such file or directory: '../output/건설사업정보시스템/품질계획 현황/selectQltPlnList.pickle'
품질계획 현황 정보 new 모드 2 


In [25]:
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]
PRIMARYKEY

'serviceKey,sptNo,rprtYm'

In [26]:
JSONKEY = "detail1"
DUMMY = 0

In [27]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
PAGEYN

0

## 품질계획 현황 전용 스크랩 함수

In [28]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os
import time


STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            time.sleep(1.2)
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

            response = requests.get(inUrl+queryParams, verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("{} page is empty".format(i))
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 detailList1~5까지 봐야 한다.
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)
                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    if(response.json()["response"]["body"].get('detailList1') != None):
                        print("we have a data on detailList1")
                        jsondataDetail = response.json()["response"]["body"]["detailList1"]

                    elif(response.json()["response"]["body"].get('detailList2') != None):
                        jsondataDetail = response.json()["response"]["body"]["detailList2"]
                        print("we have a data on detailList2")

                    elif(response.json()["response"]["body"].get('detailList3') != None):
                        jsondataDetail = response.json()["response"]["body"]["detailList3"]
                        print("we have a data on detailList3")

                    elif(response.json()["response"]["body"].get('detailList4') != None):
                        jsondataDetail = response.json()["response"]["body"]["detailList4"]
                        print("we have a data on detailList4")

                    elif(response.json()["response"]["body"].get('detailList5') != None):
                        jsondataDetail = response.json()["response"]["body"]["detailList5"]
                        print("we have a data on detailList5")
#                     print(jsondataDetail)
                    rowDataDetail = pd.DataFrame(jsondataDetail)
                    emptyPd = pd.merge(emptyPd, rowDataDetail, on='sptNo')
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

## 테스트용 코드

In [ ]:
resultDf = pd.DataFrame()
resultDfMerged = pd.DataFrame()

for i in range(0,dfCombiLen):
    sptnoValue = sptnoList[i]
    apiNo = i%APIKEYLEN
    BASEPARAM={"serviceKey":APIKEY[apiNo],"sptNo": sptnoValue, "rprtYm": "", "type":"json"}
    print("BASEPARAM ",BASEPARAM)
    resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
    print("i= ",i)
    print("resultDf ",resultDf)
    if resultDf.empty: # 정상 데이터가 없는 경우
        print("No Data")
    else:
        dateRange = pd.period_range(resultDf["stwrdt"].values[0],resultDf["ccwdt"].values[0],freq='M').strftime('%Y%m').tolist()
        for i in range(0,len(dateRange)):
            apiNo = i%APIKEYLEN
            BASEPARAM={"serviceKey":APIKEY[apiNo],"sptNo": sptnoValue, "rprtYm": str(dateRange[i]), "type":"json"}
            print("BASEPARAM ",BASEPARAM)
            resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
            resultDfMerged = resultDfMerged.append(resultDf) 


BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2018034', 'rprtYm': '', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
i=  0
resultDf        odofbrn     sptno       cno         cwknm orcd ornm rutcd  \
0  6048300784  C2018034  C2018036  의령-합천 국도건설공사    B  부산청   020   

              rutnm    pdznnm bzkdcd bzkdnm bztocd      stwrdt       ccwdt  \
0  국도 20호선(산청 ~ 포항)  경상남도 의령군      A   일반국도     01  2018-11-02  2023-10-06   

   ccwxpcdt ccwyn ccwynnm bzarcd bzarnm extcyclsecd extcyclsenm  \
0  20231006     0    진행공사    101     도로           C          시공   

          lastamt                                  cwksctnnm  index  
0  23,107,000,000  경상남도 의령군 법정로 899-1~경상남도 합천군 청덕면 앙진리 544-1      0  
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2018034', 'rprtYm': '201811', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2018

>totalCount 301
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2018034', 'rprtYm': '202201', 'type': 'json'}
1 page scraping start
>totalCount 301
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2018034', 'rprtYm': '202202', 'type': 'json'}
1 page scraping start
>totalCount 301
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2018034', 'rprtYm': '202203', 'type': 'json'}
1 page scraping start
>totalCount 301
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2018034', 'rprtYm': '202204', 'type': 'json'}
1 page scraping start
>totalCount 313
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2018034', 'rprtYm': '202205', 'type': 'json'}
1 page 

1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2010019', 'rprtYm': '201111', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2010019', 'rprtYm': '201112', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2000001', 'rprtYm': '', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
i=  2
resultDf        odofbrn     sptno       cno        cwknm orcd ornm rutcd rutnm  \
0  6048300784  C2000001  C2000001  서낙동강 하천개수공사    B  부산청    P1  서낙동강   

      pdznnm bzkdcd bzkdnm bztocd      stwrdt       ccwdt  ccwxpcdt ccwyn  \
0  부산광역시 강서구     B1    낙동강     01  2000-06-01  2010-01-03  20100103     1   

  ccwynnm bzarcd bzarnm extcyclsecd extcyclsenm         lastamt  \
0    완료공사    102     하천           C          시공  21,288,300,00

1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2000001', 'rprtYm': '200312', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2000001', 'rprtYm': '200401', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2000001', 'rprtYm': '200402', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2000001', 'rprtYm': '200403', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2000001', 'rprtYm': '200404', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2000001', 'rprtYm': 

1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2000001', 'rprtYm': '200712', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2000001', 'rprtYm': '200801', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2000001', 'rprtYm': '200802', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2000001', 'rprtYm': '200803', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2000001', 'rprtYm': '200804', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2000001', 'rprtYm': 

...

>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2005096', 'rprtYm': '201007', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2005096', 'rprtYm': '201008', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2005096', 'rprtYm': '201009', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2005096', 'rprtYm': '201010', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2005096', 'rprtYm': '201011', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2005096', 'rprtYm': '201012', 'type': 'jso

1 page scraping start
>totalCount 12
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2015028', 'rprtYm': '201705', 'type': 'json'}
1 page scraping start
>totalCount 12
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2015028', 'rprtYm': '201706', 'type': 'json'}
1 page scraping start
>totalCount 19
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2015028', 'rprtYm': '201707', 'type': 'json'}
1 page scraping start
>totalCount 20
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2015028', 'rprtYm': '201708', 'type': 'json'}
1 page scraping start
>totalCount 26
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2015028', 'rprtYm': '201709', 'type'

1 page scraping start
>totalCount 0
0 no pageNo
i=  801
resultDf        odofbrn     sptno       cno             cwknm orcd ornm rutcd rutnm  \
0  6048300784  C2016020  C2016021  남강 생초지구 하천환경정비사업    B  부산청    B1   낙동강   

     pdznnm bzkdcd bzkdnm bztocd      stwrdt       ccwdt  ccwxpcdt ccwyn  \
0  경상남도 산청군     B1    낙동강     01  2016-08-16  2020-12-02  20200725     1   

  ccwynnm bzarcd bzarnm extcyclsecd extcyclsenm         lastamt  \
0    완료공사    102     하천           C          시공  17,686,900,000   

             cwksctnnm  index  
0  산청군오부면양촌리~함양군수동면화산리      0  
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2016020', 'rprtYm': '201608', 'type': 'json'}
1 page scraping start
>totalCount 12
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2016020', 'rprtYm': '201609', 'type': 'json'}
1 page scraping start
>totalCount 12
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey

1 page scraping start
>totalCount 289
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2016020', 'rprtYm': '201911', 'type': 'json'}
1 page scraping start
>totalCount 289
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2016020', 'rprtYm': '201912', 'type': 'json'}
1 page scraping start
>totalCount 289
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2016020', 'rprtYm': '202001', 'type': 'json'}
1 page scraping start
>totalCount 289
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2016020', 'rprtYm': '202002', 'type': 'json'}
1 page scraping start
>totalCount 289
we have a data on detailList1
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2016020', 'rprtYm': '202003', '

1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2003055', 'rprtYm': '200306', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2003055', 'rprtYm': '200307', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2003055', 'rprtYm': '200308', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2003055', 'rprtYm': '200309', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'sptNo': 'C2003055', 'rprtYm': '200310', 'type': 'json'}
1 page scraping start
>totalCount 0
0 no pageNo
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'sptNo': 'C2003055', 'rprtYm': 

In [ ]:
# 쓰레드 사용하지 않는 기존 코드
# for i in range(0,dfCombiLen):
#     sptnoValue = dfCombi.loc[i].sptno
#     BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue, "rprtYm": "", "type":"json"}
#     resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
#     print("i= ",i)
#     if resultDf.empty: # 정상 데이터가 없는 경우
#         print("No Data")
#     else:
# #         tmpStartDay = resultDf["stwrdt"].values[0].split('-')
# #         tmpEndDay = resultDf["ccwdt"].values[0].split('-')

#         dateRange = pd.period_range(resultDf["stwrdt"].values[0],resultDf["ccwdt"].values[0],freq='M').strftime('%Y%m').tolist()
#         print(dateRange)
#         for i in range(0,len(dateRange)):
#             BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue, "rprtYm": str(dateRange[i]), "type":"json"}
#             resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
#             resultDfMerged = resultDfMerged.append(resultDf) 
# #             print("----------------------------------------")
# #             print(resultDf)
        

In [37]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

품질계획 현황 save compled


In [38]:
resultDfMerged.shape

(108, 33)

### [추가] 파라미터 정보 추가저장

In [39]:
# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
sptnoList = orgParam + sptnoList

# 최종 파라미터 저장
cf.saveparam(sptnoList, SITENAME,DATANAME,SERVICENAME)